In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [54]:
data = pd.read_csv("./sample_data/nutrition.csv")
data['name'] = data['name'].str.split(', ')
data = data.explode('name')
# Reset the index
data.reset_index(drop=True, inplace=True)

In [55]:
selected_features = ['calories', 'total_fat', 'saturated_fat', 'cholesterol', 'sodium',
                     'protein', 'carbohydrate', 'fiber', 'sugars']
columns_to_drop = [col for col in data.columns if col not in selected_features]
data = data.drop(columns=columns_to_drop)
data = data.dropna(how='all')

In [ ]:
postfix_to_grams = {
    'mg': 0.001,
    'g': 1,
    'mcg': 0.000001,
}
for column in data.columns:
    for index, value in enumerate(data[column]):
        if isinstance(value, str):
            for postfix, conversion_factor in postfix_to_grams.items():
                if postfix in value:
                    try:
                        # Extract the numeric part of the value and convert it to grams
                        numeric_value = float(value.replace(postfix, '').strip())
                        converted_value = numeric_value * conversion_factor
                        data.at[index, column] = converted_value
                    except ValueError:
                        # Handle cases where the conversion fails
                        print(f"Error converting value: {value} in column: {column} at index: {index}")


In [57]:
data = data.fillna(0)
data[selected_features] = data[selected_features].apply(pd.to_numeric, errors='coerce')
threshold_factors = {
    'calories': 2,  # Lowering calorie intake can help with weight management
    'total_fat': 1.5,  # Limiting fat intake can help control overall calorie and fat consumption
    'saturated_fat': 1,  # Limiting saturated fat is recommended for heart health
    'cholesterol': 1,  # Limiting cholesterol intake is recommended for heart health
    'sodium': 1,  # Controlling sodium intake is important for managing blood pressure
    'protein': 1.5,  # Adequate protein intake can help with satiety and muscle maintenance
    'carbohydrate': 1.5,  # Monitoring carbohydrate intake is essential for blood sugar control
    'fiber': 2,  # Increasing fiber intake can help stabilize blood sugar levels
    'sugars': 1.5,  # Limiting added sugars is important for managing blood sugar levels
}

In [58]:
nutrient_stats = data[selected_features].describe().loc[['mean', 'std']]

In [59]:
thresholds = {}
nutrient_columns = selected_features
for nutrient in nutrient_columns:
    threshold = nutrient_stats.loc['mean', nutrient] + threshold_factors.get(nutrient, 0) * nutrient_stats.loc['std', nutrient]
    thresholds[nutrient] = threshold

In [60]:
data['edible_for_diabetes'] = 1
for nutrient in nutrient_columns:
    data['edible_for_diabetes'] &= (data[nutrient] <= thresholds[nutrient])

In [62]:
X = data[selected_features]
y = data['edible_for_diabetes']

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [64]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [66]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)

RandomForestClassifier(random_state=42)